In [ ]:
ticker = yf.Ticker("AAPL")
chain = ticker.option_chain(ticker.options[-1])  # farthest expiry
calls = chain.calls

row = calls.iloc[0]  # first row of calls DataFrame

S = ticker.history(period="1d")["Close"].iloc[-1]   
K = np.array(calls.iloc[:,2])                              
expiry = pd.to_datetime(ticker.options[-1])        
today = pd.Timestamp.today().normalize()
T = (expiry - today).days / 365                    
r = 0.03                                         
price = (calls.iloc[:,4]+calls.iloc[:,5])/2;
impV = np.array(calls.iloc[:,10])       

div = ticker.dividends.tz_localize(None)
ttm = div[div.index >= pd.Timestamp.today().normalize()-pd.Timedelta(days=365)].sum()
q_ttm = float(ttm / S) 

In [ ]:
res= []
for i in range(0,len(K)):
    res.append(bs.implied_vol(S, K[i] ,r, q_ttm, T, 'call',price[i]))

res = np.array(res)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
mask = np.isfinite(res) & np.isfinite(impV) & (impV > 1e-4)
Kp, resp, impVp = K[mask], res[mask], impV[mask]
# Plot
ax.plot(Kp, resp, marker='o', ms=3, lw=2.0, color='#1f77b4', label='Model')
ax.plot(Kp, impVp, marker='o', ms=3, lw=2.0, color='#ff7f0e', label='Market (yfinance)')

# ATM line
ax.axvline(S, ls='--', lw=1, color='gray', alpha=0.7)
ax.text(S+2, ax.get_ylim()[1]*0.95, "ATM", color='gray')

# Labels
ax.set_title("AAPL Implied Volatility Smile", fontsize=14, weight='bold')
ax.set_xlabel("Strike Price (K)", fontsize=12)
ax.set_ylabel("Implied Volatility", fontsize=12)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Legend
ax.legend(frameon=False, fontsize=11, loc="upper right")

# Grid
ax.grid(True, alpha=0.3)

# Tight layout
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))

plt.plot(Kp, np.abs(1 - resp/impVp), 
         marker='o', markersize=4, linewidth=1.2, 
         color='darkred', label='Relative Error vs yfinance IV')

# Labels and title
plt.xlabel("Strike Price (K)", fontsize=12)
plt.ylabel("Relative Error", fontsize=12)
plt.title("Validation of Implied Volatility Calculation", fontsize=14, weight="bold")

# Log scale for error makes sense here
plt.yscale('log')
plt.ylim(1e-2, 0.6) 

# Grid + legend
plt.grid(True, which="both", linestyle="--", alpha=0.6)
plt.legend(fontsize=11)

plt.tight_layout()
plt.show()


